Dual-mixed formulation for Poisson equation
======================================

This demo is implemented in a single Python file,
demo\_mixed-poisson-dual.py, which contains both the variational forms
and the solver.

# Implementation

This demo is implemented in the demo\_mixed-poisson-dual.py file.

First, the :pydolfin module is imported:

In [ ]:
from dolfin import *

Then, we need to create a :pyMesh &lt;dolfin.cpp.Mesh&gt; covering the
unit square. In this example, we will let the mesh consist of 32 x 32
squares with each square divided into two triangles:

In [ ]:
# Create mesh
mesh = UnitSquareMesh(32, 32)

Next, we need to define the function spaces. We define the two function
spaces $\Sigma_h = DRT$ and $V_h = CG$ separately, before combining
these into a mixed function space:

In [ ]:
# Define function spaces and mixed (product) space
DRT = FunctionSpace(mesh, "DRT", 2)
CG = FunctionSpace(mesh, "CG", 3)
W = DRT * CG

The second argument to :pyFunctionSpace
&lt;dolfin.functions.functionspace.FunctionSpace&gt; specifies the type
of finite element family, while the third argument specifies the
polynomial degree. The UFL user manual contains a list of all available
finite element families and more details. The \* operator creates a
mixed (product) space `W` from the two separate spaces `DRT` and `CG`.
Hence,

$$W = \{ (\tau, v) \ \text{such that} \ \tau \in DRT, v \in CG \}.$$

Next, we need to specify the trial functions (the unknowns) and the test
functions on this space. This can be done as follows

In [ ]:
# Define trial and test functions
(sigma, u) = TrialFunctions(W)
(tau, v) = TestFunctions(W)

In order to define the variational form, it only remains to define the
source functions $f$ and $g$. This is done just as for the
mixed Poisson demo
&lt;demo\_pde\_mixed-poisson\_python\_documentation&gt;:

In [ ]:
# Define source functions
f = Expression("10*exp(-(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2)) / 0.02)")
g = Expression("sin(5.0*x[0])")

We are now ready to define the variational forms a and L.

In [ ]:
# Define variational form
a = (dot(sigma, tau) + dot(grad(u), tau) + dot(sigma, grad(v)))*dx
L = - f*v*dx - g*v*ds

It only remains to prescribe the Dirichlet boundary condition for $u$.
Essential boundary conditions are specified through the class
:pyDirichletBC &lt;dolfin.fem.bcs.DirichletBC&gt; which takes three
arguments: the function space the boundary condition is supposed to be
applied to, the data for the boundary condition, and the relevant part
of the boundary.

We want to apply the boundary condition to the second subspace of the
mixed space. Subspaces of a :pyMixedFunctionSpace
&lt;dolfin.functions.functionspace.MixedFunctionSpace&gt; can be
accessed by the method :pysub
&lt;dolfin.functions.functionspace.FunctionSpaceBase.sub&gt;. In our
case, this reads `W.sub(1)`. (Do *not* use the separate space `CG` as
this would mess up the numbering.)

Specifying the relevant part of the boundary can be done as for the
Poisson demo:

In [ ]:
# Define Dirichlet BC
def boundary(x):
    return x[0] < DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS

Now, all the pieces are in place for the construction of the essential
boundary condition:

In [ ]:
bc = DirichletBC(W.sub(1), 0.0, boundary)

To compute the solution we use the bilinear and linear forms, and the
boundary condition, but we also need to create a :pyFunction
&lt;dolfin.functions.function.Function&gt; to store the solution(s). The
(full) solution will be stored in the `w`, which we initialise using the
:pyFunctionSpace
&lt;dolfin.functions.functionspace.FunctionSpace&gt; `W`. The actual
computation is performed by calling :pysolve
&lt;dolfin.fem.solving.solve&gt;. The separate components `sigma` and
`u` of the solution can be extracted by calling the :pysplit
&lt;dolfin.functions.function.Function.split&gt; function. Finally, we
plot the solutions to examine the result.

In [ ]:
# Compute solution
w = Function(W)
solve(a == L, w, bc)
(sigma, u) = w.split()

# Plot sigma and u
plot(sigma)
plot(u)
interactive()

# Complete code